In [2]:
import tensorflow as tf

D:\anaconda\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


KeyboardInterrupt: 

In [ ]:
import scipy.io as sio
import numpy as np
'''keywords = ['blink','event','fixation','PD','pupil','saccade']
dataset_path = dict()
dataset = dict()
for k in keywords:
    dataset_path[k] = "D:/Downloads/SEED_IV/eye_raw_data/"+"huan_20150915_"+k+".mat"
    temp = sio.loadmat(dataset_path[k])
    for x in temp:
        if isinstance(temp[x],np.ndarray):
            dataset[k] = temp[x] 


print('blink: ',dataset['blink'][1,0].shape)   # 
print('event: ',dataset['event'][1,0].shape)   # 
print('fixation: ',dataset['fixation'][1,0].shape)   # 
print('sum: ', np.sum(dataset['fixation'][0,0], axis=0))
print('pupil: ',dataset['pupil'][1,0][:10])      # 
print('PD: ',dataset['PD'][1,0].shape)         # pupil diameter
print('saccade: ',dataset['saccade'][1,0].shape)    '''     

In [ ]:
'''import os
dataset_dir		=   "./data_preprocessed_matlab/"
window_size		=	128
output_dir		=   "./eye/"
# get directory name for one subject
record_list = [task for task in os.listdir(dataset_dir) if os.path.isfile(os.path.join(dataset_dir,task))]
eye_data = []
label_data = []
for record in record_list:
    file = os.path.join(dataset_dir,record)
    print(file)
    data = sio.loadmat(file)['data']
    labels = sio.loadmat(file)['labels']
    eye_data.append(data[:,32:34,:])
    label_data.append(labels)
eye_data = np.array(eye_data)
label_data = np.array(label_data)'''

In [ ]:
def windows(data, size):
    start = 0
    while ((start+size) < data.shape[0]):
        yield int(start), int(start + size)
        start += size

In [ ]:
def segment_signal_without_transition(data,label,label_index,window_size):
    # get data file name and label file name
    for (start, end) in windows(data, window_size):
        if((len(data[start:end]) == window_size)):
            if(start == 0):
                segments = data[start:end]
                segments = np.vstack([segments, data[start:end]])
                
                labels = np.array(label[label_index])
                labels = np.append(labels, np.array(label[label_index]))
            else:
                segments = np.vstack([segments, data[start:end]])
                labels = np.append(labels, np.array(label[label_index])) # labels = np.append(labels, stats.mode(label[start:end])[0][0])
    return segments, labels

In [ ]:
# 对一维特征进行特征标准化
def feature_normalize(data):
    # 计算样本均值
    mean = data[data.nonzero()].mean()
    # 计算样本标准差
    sigma = data[data. nonzero ()].std()
    data_normalized = data
    data_normalized[data_normalized.nonzero()] = (data_normalized[data_normalized.nonzero()] - mean)/sigma
    # return shape: 9*9
    return data_normalized

In [ ]:
import pickle
import time
begin = time.time()
print("time begin:",time.localtime())
window_size = 128
output_dir		=   "../preprocesseddata/"
suffix          =   'no'     # yes/no (using baseline signals or not)
output_dir = output_dir+suffix+"_"+"/"
if os.path.isdir(output_dir)==False:
    os.makedirs(output_dir)
eye_data = eye_data.transpose(0,1,3,2)
eye_data_aft_proc = np.zeros(shape=[eye_data.shape[0],eye_data.shape[1],60,window_size,eye_data.shape[-1]])
for person in range(32):
    # label只选择arousal
    L = label_data[person,:,0].reshape(-1,1)
    ed = eye_data[person]
    for i,video in enumerate(ed):
        # 不用base,去掉前三个mj  
        video_h = video[3 * 128:,0]
        video_v = video[3 * 128:,1]
        video_h = feature_normalize(video_h)
        video_v = feature_normalize(video_v)
        video_h, L = segment_signal_without_transition(video_h,L,i,window_size)
        video_v, L = segment_signal_without_transition(video_v,L,i,window_size)
        index = np.array(range(video_h.shape[0]))
        np.random.shuffle(index)
        video_h = video_h[index]
        video_v = video_v[index]
        eye_data_aft_proc[person,i,:,:,0] = video_h
        eye_data_aft_proc[person,i,:,:,1] = video_v
print(eye_data_aft_proc.shape)
for i in range(32):
    output_file = output_dir+"AFT_PREPROCESS_"+str(i+1)+"_DATASET.pkl"
    output_label = output_dir+"AFT_PREPROCESS_"+str(i+1)+"_LABELSET.pkl"
    with open(output_file, "wb") as fp:
        pickle.dump(eye_data_aft_proc[i] ,fp, protocol=4)
    with open(output_label, "wb") as fp:
        pickle.dump(label_data[i], fp, protocol=4)
    end = time.time()
    print("end time:",time.localtime())
    print("time consuming:",(end-begin))

In [56]:
import pandas as pd
labels = np.array([1,2,4,3,5])
labels = list(labels)
one_hot_labels = np.array(list(pd.get_dummies(labels)))
labels = np.asarray(pd.get_dummies(labels), dtype=np.int8)
print(labels)

[[1 0 0 0 0]
 [0 1 0 0 0]
 [0 0 0 1 0]
 [0 0 1 0 0]
 [0 0 0 0 1]]


In [62]:
a = np.array([1,2,3,4,5]).reshape(5,1)
a.flatten()
print(a.shape)
[x for x in a[:,0]]

(5, 1)


[1, 2, 3, 4, 5]